In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Integer, Categorical
from sklearn.metrics import r2_score, mean_absolute_error, make_scorer
from sklearn.metrics import mean_absolute_percentage_error

In [6]:
data = pd.read_csv("veri_EURTRY.csv")

In [7]:
data.columns = ["Close", "High", "Low", "Open", "Day", "Month", "Year"]

In [8]:
data = data.dropna()
X = data[["Open", "High", "Low", "Day", "Month", "Year"]]
y = data["Close"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
param_grid = {
    "n_estimators": Integer(50, 300),
    "max_depth": Integer(5, 20),
    "min_samples_split": Integer(2, 10),
    "min_samples_leaf": Integer(1, 10),
    "max_features": Categorical(["sqrt", "log2", None])
}


In [11]:
scorer = make_scorer(r2_score)

In [12]:
ga_search = GASearchCV(
    estimator=ExtraTreesRegressor(random_state=42),
    param_grid=param_grid,
    scoring=scorer,
    cv=3,
    verbose=True,
    population_size=10,
    generations=5,
    n_jobs=-1,
    keep_top_k=3,
    error_score='raise'
)

In [13]:
ga_search.fit(X_train, y_train)

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.999822	0.000139102	0.999956   	0.9995     
1  	20    	0.999943	1.70427e-05	0.999971   	0.999924   
2  	20    	0.999961	7.04638e-06	0.999971   	0.999953   
3  	20    	0.999966	4.99658e-06	0.999971   	0.999955   
4  	20    	0.999966	3.53014e-06	0.999971   	0.999961   
5  	20    	0.999967	2.21792e-06	0.999969   	0.999962   


GASearchCV(error_score='raise',
           estimator=ExtraTreesRegressor(max_depth=19, max_features=None,
                                         n_estimators=176, random_state=42),
           generations=5, keep_top_k=3, n_jobs=-1,
           param_grid={'max_depth': <sklearn_genetic.space.space.Integer object at 0x000001FC1370C810>,
                       'max_features': <sklearn_genetic.space.space.Categorical object at 0x000001FC1370CB10>,
                       'min_samples_leaf': <sklearn_genetic.space.space.Integer object at 0x000001FC1370CA50>,
                       'min_samples_split': <sklearn_genetic.space.space.Integer object at 0x000001FC1370C990>,
                       'n_estimators': <sklearn_genetic.space.space.Integer object at 0x000001FC133BDF10>},
           population_size=10, return_train_score=True,
           scoring=make_scorer(r2_score, response_method='predict'),
           warm_start_configs=[])

In [16]:
y_pred = ga_search.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)


In [17]:
print("R²:", round(r2, 4))
print("MAE:", round(mae, 4))
print(f"Test MAPE: {mape:.4f}")
print("En iyi hiperparametreler:", ga_search.best_params_)

R²: 1.0
MAE: 0.0093
Test MAPE: 0.0021
En iyi hiperparametreler: {'n_estimators': 176, 'max_depth': 19, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}
